In [1]:
import numpy as np
import sympy as sp

In [2]:
class FiniteField:
    def __init__(self, value, prime):
        self.value = value % prime
        self.prime = prime
    
    def __add__(self, other: 'FiniteField') -> 'FiniteField':
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value + other.value, self.prime)
    
    def __mul__(self, other: 'FiniteField') -> 'FiniteField':
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value * other.value, self.prime)
    
    def __neg__(self) -> 'FiniteField':
        return FiniteField(-self.value, self.prime)
    
    def __truediv__(self, other: 'FiniteField') -> 'FiniteField':
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value * pow(other.value, -1, self.prime), self.prime)
    
    def __repr__(self):
        return f"{self.value} (mod {self.prime})"
    

In [3]:
class UnivariatePolynomial:
    def __init__(self, coeffcients, symbol='x'):
        self.coefficients = coeffcients
        self.symbol = symbol
        
    def initialize(self):
        x = sp.symbols(self.symbol)
        # 딕셔너리를 이용해 다항식 생성
        return sp.expand(sum(coeff * x**exp for exp, coeff in self.coefficients.items()))  
    


In [4]:
def lagrange_interpolation(points, prime):
    x = sp.symbols('x')
    n = len(points)    
    poly= 0
    
    for i in range(n):
        lj_basis = 1
        for j in range(n):
            if i != j:
                lj_basis *= (x - points[j][0]) * pow(points[i][0] - points[j][0], -1, prime)
        poly += lj_basis * points[i][1]    
                   
    return sp.expand(poly) % prime

In [5]:
points = [(1, 2), (2, 3), (3, 5)]
result = lagrange_interpolation(points, 7)
result

Mod(74*x**2 - 312*x + 2, 7)

In [6]:
result.subs('x', 3)

5

In [7]:
coeffs = {
    -2: 2,  # 2/x
    1: 3,   # 3x
    0: -5   # -5
}
poly = UnivariatePolynomial(coeffs)
result = poly.initialize()
result

3*x - 5 + 2/x**2

In [8]:
def find_generators(prime):
    prime_factors = sp.factorint(prime-1)
    
    for g in range(2, prime):
        if all(pow(g, (prime-1)//factor, prime) != 1 for factor in prime_factors.keys()):
            return g
    return None

In [9]:
def find_nth_root_of_unity(prime, n):
    g = find_generators(prime)
    return pow(g, (prime-1)//n, prime)

In [10]:
class EllipticCurve:
    def __init__(self, a, b, prime):
        self.a = a
        self.b = b
        self.prime = prime        
        self.discriminant = FiniteField(4 * a**3 + 27 * b**2, prime)
        if self.discriminant.value == 0:
            raise ValueError("This curve is not elliptic")
        
        self.poly = UnivariatePolynomial({3: 1, 1: self.a, 0: self.b}).initialize() % prime
    
    def is_point_on_curve(self, x, y):
        if x is None and y is None:  # 무한점 처리
            return True
        return (y * y) % self.prime == self.poly.subs('x', x)

    def generate_points(self):
        points = []
        for x in range(self.prime):
            rhs = self.poly.subs('x', x)
            for y in range(self.prime):
                if (y * y) % self.prime == rhs:
                    points.append((x, y))
        return points

In [11]:
ecc = EllipticCurve(3, 5, 7)
points = ecc.generate_points()
points

[(1, 3), (1, 4), (4, 2), (4, 5), (6, 1), (6, 6)]

In [12]:
ecc.is_point_on_curve(4,5)

True

In [13]:
class EllipticCurvePoint:
    def __init__(self, x, y, curve: EllipticCurve):
        self.x = x
        self.y = y
        self.curve = curve
        if not curve.is_point_on_curve(x, y):
            raise ValueError("Point is not on the curve")
    
    def __add__(self, other):
        if self.x is None:  # 무한점 처리
            return other
        if other.x is None:
            return self

        if self.x == other.x:
            if self.y == -other.y:  # P + (-P) = 무한점
                return EllipticCurvePoint(None, None, self.curve)
            else:  # 점 두 배
                steep = (3 * self.x**2 + self.curve.a) * pow(2 * self.y, -1, self.curve.prime) % self.curve.prime
        else:  # 서로 다른 두 점
            steep = (other.y - self.y) * pow(other.x - self.x, -1, self.curve.prime) % self.curve.prime

        result_x = (steep**2 - self.x - other.x) % self.curve.prime
        result_y = (steep * (self.x - result_x) - self.y) % self.curve.prime

        return EllipticCurvePoint(result_x, result_y, self.curve)
        
    def __mul__(self, n):
        result = EllipticCurvePoint(None, None, self.curve)  # 무한점으로 초기화
        temp = self  # 현재 점

        while n > 0:
            if n & 1:  # 마지막 비트가 1이면 현재 점을 결과에 더함
                result = result + temp
            temp = temp + temp  # 점 두 배
            n >>= 1  # n을 오른쪽으로 비트 이동 (n // 2)
        
        return result

In [17]:
P = EllipticCurvePoint(points[0][0], points[0][1], ecc)
Q = EllipticCurvePoint(points[2][0], points[2][1], ecc)
print(points[0], points[2])
ecc.is_point_on_curve(P.x,P.y), ecc.is_point_on_curve(Q.x,Q.y)
R = P + P + P

(1, 3) (4, 2)


In [18]:
R.x, R.y

(4, 5)

In [19]:
M = P * 3
M.x, M.y

(4, 5)

In [134]:
def generate_srs(generator, tau, n, prime):
    return [pow(generator, tau**i, prime) for i in range(n)]

def evaluate(gp, coeffs, u, prime):
    # 다항식 생성
    polynomial = UnivariatePolynomial(coeffs).initialize()
    f_u = polynomial.subs('x', u) % prime

    # 나눗셈 후 단순화
    numerator = polynomial - f_u
    denominator = UnivariatePolynomial({1: 1, 0: -u}).initialize()
    q_x = (numerator / denominator) % prime

    # 커밋먼트 증명 생성
    pi = 1
    for i, coeff in enumerate([coeffs[key] for key in sorted(coeffs.keys())]):  # 정렬된 순서로 계수 사용
        pi *= pow(gp[i], int(coeff), prime)
        pi %= prime  # 안전하게 모듈로 연산
    
    return f_u, pi

In [135]:
evaluate(gp, coeffs, 11, prime)

(42, 38)

In [136]:
prime = 101
tau = 13
generator = find_generators(prime)

coeffs = {
    0: 2,    
    1: 2,
    3: 1    
}

gp = generate_srs(generator, tau, len(coeffs), prime)

polynomial = UnivariatePolynomial(coeffs).initialize() % prime
commitment = pow(generator, int(polynomial.subs('x', tau)), prime)

In [137]:
list(coeffs.keys())

[0, 1, 3]

In [138]:
evaluate(gp, coeffs, 11, prime)

(42, 38)

In [139]:
evaluate(gp, coeffs, 11, prime)

(42, 38)